In [ ]:
!pip install nb_black -q

In [ ]:
%load_ext nb_black

# The dataset

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pandas_datareader.data as web
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

![](https://upload.wikimedia.org/wikipedia/commons/b/bd/Tesla_Motors.svg)




Tesla, Inc. (formerly Tesla Motors, Inc.) is an American electric vehicle and clean energy company based in Palo Alto, California. Tesla's current products include electric cars (the Model S, Model 3, Model X, and Model Y), battery energy storage from home to grid scale (the Powerwall, Powerpack, and Megapack), solar products (solar panels and solar roof tiles) and related products and services.

[The dataset!](https://www.kaggle.com/rpaguirre/tesla-stock-price)

Source - [Wikipedia](https://en.wikipedia.org/wiki/Tesla,_Inc.)

In [ ]:
data = web.DataReader("TSLA", data_source="yahoo", start="2010-06-29", end="2020-11-25")
data = data.round(3)
data.head()

Just take a look at time serie.

In [ ]:
fig = px.line(
    data.reset_index(), x="Date", y="Close", title="Tesla Stock Price - Close"
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

# T-delay

Creating the dataset to be used by RNN.

<img src="https://www.tensorflow.org/tutorials/structured_data/images/split_window.png" align="left"  />

In [ ]:
def create_dataset(dataset, look_back=60):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i : (i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return pd.DataFrame(np.array(dataX)), np.array(dataY)


X_close, Y_close = create_dataset(data[["Close"]].values)
aux = X_close
aux["Target"] = Y_close
aux

In [ ]:
del aux

## Reshaping
The dimensions is (n_sample, t_delays, 1).

Divide between train and test.


In [ ]:
def split_dataset(X, Y):
    X_train = X[:-100].values
    y_train = Y[:-100]

    X_test = X[-100:].values
    y_test = Y[-100:]
    return X_train, y_train, X_test, y_test


# Close dataset
X_train_close, y_train_close, X_test_close, y_test_close = split_dataset(
    X_close, Y_close
)

Using MixMaxScaler on feature.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc_close_x = MinMaxScaler()
sc_close_y = MinMaxScaler()

# X Scale
X_train_close_scaled = sc_close_x.fit_transform(X_train_close).reshape(
    *X_train_close.shape, 1
)
X_test_close_scaled = sc_close_x.transform(X_test_close).reshape(*X_test_close.shape, 1)

# Y Scale
y_train_close_scaled = sc_close_y.fit_transform(y_train_close.reshape(-1, 1)).reshape(
    -1
)
y_test_close_scaled = sc_close_y.transform(y_test_close.reshape(-1, 1)).reshape(-1)

In [ ]:
print(X_train_close_scaled.shape)
print(y_train_close_scaled.shape)

# RNN
Creating and traing the RNN.

<br/>
<img src="https://miro.medium.com/max/770/1*xTKE0g6XNMLM8IQ4aFdP0w.png" width='800px' align="left" />

In [ ]:
%%time 

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

def gen_model():
    rnn = Sequential()
    rnn.add(LSTM(units=360, return_sequences=True, input_shape=(X_train_close_scaled.shape[1], 1)))
    rnn.add(Dropout(0.2))
    rnn.add(LSTM(units=360, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(LSTM(units=360, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(LSTM(units=360, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(LSTM(units=360, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(LSTM(units=360))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=1,activation='relu'))
    rnn.compile(optimizer="adam", loss=["mean_squared_error",'cosine_similarity'])
    callback = keras.callbacks.EarlyStopping(
        monitor="val_loss",
        min_delta=0,
        patience=0,
        verbose=False,
        mode="auto",
        baseline=None,
        restore_best_weights=False,
    )
    return rnn

rnn = gen_model()
history = rnn.fit(X_train_close_scaled, y_train_close_scaled,validation_data=(X_test_close_scaled,y_test_close_scaled), epochs=40, batch_size=32)

Predictions and take the real values.

In [ ]:
history_df = pd.DataFrame(history.history)
go.Figure() \
.add_trace(go.Scatter(x=history_df.index, y=history_df.loss, mode="lines", name="Train Loss")) \
.add_trace(go.Scatter(x=history_df.index, y=history_df.val_loss, mode="lines", name="Validation Loss")) \
.update_layout(
    title="Train Metrics",
    xaxis_title="Epochs",
    yaxis_title="Loss",
)


# Plot results and evaluation
Just using R2 and MSE to mensure the RNN performace and ploting the graph to compare predict values and real values as well.

In [ ]:
# Model
y_pred = rnn.predict(X_test_close_scaled)
y_true = y_test_close_scaled
test_dates = data.reset_index()["Date"][-100:]

# Original values
y_pred_original = sc_close_y.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1)
y_true_original = sc_close_y.inverse_transform(y_true.reshape(-1, 1)).reshape(-1)

In [ ]:
# Create traces
go.Figure().add_trace(
    go.Scatter(x=test_dates, y=y_true_original, mode="lines", name="Real Close",)
).add_trace(
    go.Scatter(x=test_dates, y=y_pred_original, mode="lines", name="Predicted Close",)
).update_layout(
    title="Stock price - Tesla Motors",
    xaxis_title="Date",
    yaxis_title="Stock price ($)",
).show()

print("r2_score close :", r2_score(y_true, y_pred))
print("mean_squared_error close:", mean_squared_error(y_true, y_pred))

The predicted curve looks more smooth than real, lika a 'trend'.